In [23]:
%run 01_hash.ipynb

In [24]:
from copy import deepcopy

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [05_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [25]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.ctime()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop == 'sig': self.signed = True
    
    def smry(self): return f'{pmh(self.fr)} -> {pmh(self.to)} {self.value} eth'

Create random txs.

In [26]:
tx1 = TX(rh(), rh(), 12.0, 0.2,  0)
tx2 = TX(rh(), rh(), 6.0,  0.15, 0); print(tx1)

fr:            📾 0x98cffe36dc...69c
hash:          📦 0x80c95f6ada...9b7
to:            📀 0x5aaab7d340...357
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 15:15:32 2021
signed:        False


#### TX Summary

In [27]:
tx1.smry()

'📾 0x98cf -> 📀 0x5aaa 12.0 eth'

### TX JSON

Every tx has a json representation that we will use for our API later.

In [28]:
tx1_json = tx1.json(); print(tx1_json)

{
    "fr": "0x98cffe36dc311f41674d7b40fab407fcb2a3b16cdb5596f8a6e2f86034c8169c",
    "hash": "0x80c95f6ada198ed080df3ec72310fd43eb19203a9740bf59db6a68c4b9fe99b7",
    "to": "0x5aaab7d340e4205c0756fb14abcae67508685220b15e34a0e59e5824c6a2e357",
    "value": 12.0,
    "fee": 0.2,
    "nonce": 0,
    "time": "Tue Apr  6 15:15:32 2021",
    "signed": false
}


We can load this JSON string `d` and create a tx object.

In [29]:
def load_tx(d):
    d = json.loads(d)
    tx = TX(d['fr'],d['to'],d['value'],d['fee'],d['nonce'])
    for k,v in d.items(): setattr(tx, k, v)
    tx.hash = d['hash']
    return tx

In [30]:
tx1_from_json = load_tx(tx1_json)
assert tx1 == tx1_from_json

#### Print several txs in a nice way.

In [31]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            📾 0x98cffe36dc...69c
hash:          📦 0x80c95f6ada...9b7
to:            📀 0x5aaab7d340...357
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 15:15:32 2021
signed:        False

fr:            💹 0x532666a841...ab2
hash:          📓 0x6dd5970374...686
to:            📢 0x7c4d6916e9...dfc
value:         6.0 eth
fee:           0.15 eth
nonce:         0
time:          Tue Apr  6 15:15:32 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [33]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            💹 0x532666a841...ab2
hash:          🔧 0xc1a96432c9...ff6
to:            📢 0x7c4d6916e9...dfc
value:         120
fee:           0.15 eth
nonce:         0
time:          Tue Apr  6 15:15:32 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [34]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [35]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            📾 0x98cffe36dc...69c
hash:          📦 0x80c95f6ada...9b7
to:            📀 0x5aaab7d340...357
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Tue Apr  6 15:15:32 2021
signed:        True
sig:           📍 0x6730e5e62d...a37


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [36]:
assert tx1 == tx1_signed